<a href="https://colab.research.google.com/github/Helen-Han0802/Tutorial-Material/blob/master/Explained_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [22]:
!rm -rf bert
!git clone https://github.com/google-research/bert
import sys
sys.path.append('bert/')
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import codecs
import collections
import json
import re
import os
import pprint
import numpy as np
import tensorflow as tf
import modeling
import tokenization

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 328.28 KiB | 4.26 MiB/s, done.
Resolving deltas: 100% (182/182), done.


clone the official BERT repository, add its directory to the path and import the appropriate modules



In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
with open('/content/drive/MyDrive/reviews_Musical_Instruments.json', 'r') as f:
  lines = f.readlines()
  print(lines[0])
  docs = [json.loads(line)['reviewText'] for line in lines]
print(docs[:5])

{"reviewerID": "A1YS9MDZP93857", "asin": "0006428320", "reviewerName": "John Taylor", "helpful": [0, 0], "reviewText": "The portfolio is fine except for the fact that the last movement of sonata #6 is missing. What should one expect?", "overall": 3.0, "summary": "Parts missing", "unixReviewTime": 1394496000, "reviewTime": "03 11, 2014"}

['The portfolio is fine except for the fact that the last movement of sonata #6 is missing. What should one expect?', 'If you are a serious violin student on a budget, this edition has it all: Piano accompaniment, low price, urtext solo parts, and annotations by Maestro David Oistrakh where Bach\'s penmanship is hard to decipher.  Additions (in dashes) are easily distinguishable from the original bowings.  This is a delightful concerto that all intermediate level violinists should perform with a violin buddy.  Get your copy, today, along with "The Green Violin; Theory, Ear Training, and Musicianship for Violinists" book to prepare for this concerto and

In [5]:
print(len(docs))

500176


In [24]:
subdoc = docs[0:100000]

In [23]:
print(subdoc[100000])

IndexError: ignored

The two modules imported from BERT are modeling and tokenization. Modeling includes the BERT model implementation and tokenization is for tokenizing the sequences.

Fetch our TPU address from colab and initialize a new tensorflow session. **(This only applies for Colab. When running locally, it is not needed**). 

In [8]:
assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()

with tf.compat.v1.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.29.32.34:8470
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12860872434307174269),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17733532426205950243),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 13635877219152154152),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16585347685389070969),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 2672155547170345382),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0

## select BERT model
L (transformer blocks) H (hidden layer size) A (attention heads) 

In [9]:
## select BERT model
# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


In [10]:
## define global parameters
LAYERS = [-1,-2,-3,-4]
NUM_TPU_CORES = 8
MAX_SEQ_LENGTH = 128
BERT_CONFIG = BERT_PRETRAINED_DIR + '/bert_config.json'
CHKPT_DIR = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
VOCAB_FILE = BERT_PRETRAINED_DIR + '/vocab.txt'
INIT_CHECKPOINT = BERT_PRETRAINED_DIR + '/bert_model.ckpt'
BATCH_SIZE = 128

In [11]:
class InputExample(object):

  def __init__(self, unique_id, text_a, text_b=None):    
    self.unique_id = unique_id
    self.text_a = text_a
    self.text_b = text_b

  #set text_b to None by default when use single sequences rather than a sequence-pairs.

In [12]:
class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self, unique_id, tokens, input_ids, input_mask, input_type_ids):
    self.unique_id = unique_id
    self.tokens = tokens                     #a vector of input tokens
    self.input_ids = input_ids               #the ids that correspond to the token from the vocabulary
    self.input_mask = input_mask             #annotates real token sequence from padding
    self.input_type_ids = input_type_ids     #separates segment A from segment B 


In [13]:
def input_fn_builder(features, seq_length):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_unique_ids = []
  all_input_ids = []
  all_input_mask = []
  all_input_type_ids = []

  for feature in features:
    all_unique_ids.append(feature.unique_id)
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_input_type_ids.append(feature.input_type_ids)

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    num_examples = len(features)

    # This is for demo purposes and does NOT scale to large data sets. We do
    # not use Dataset.from_generator() because that uses tf.py_func which is
    # not TPU compatible. The right way to load data is with TFRecordReader.
    d = tf.data.Dataset.from_tensor_slices({
        "unique_ids":
            tf.constant(all_unique_ids, shape=[num_examples], dtype=tf.int32),
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_type_ids":
            tf.constant(
                all_input_type_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
    })

    d = d.batch(batch_size=batch_size, drop_remainder=False)
    return d

  return input_fn
  
def model_fn_builder(bert_config, init_checkpoint, layer_indexes, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    input_type_ids = features["input_type_ids"]

    model = modeling.BertModel(
        config=bert_config,
        is_training=False,
        input_ids=input_ids,
        input_mask=input_mask,
        token_type_ids=input_type_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    if mode != tf.estimator.ModeKeys.PREDICT:
      raise ValueError("Only PREDICT modes are supported: %s" % (mode))

    tvars = tf.trainable_variables()
    scaffold_fn = None
    (assignment_map,
     initialized_variable_names) = modeling.get_assignment_map_from_checkpoint(
         tvars, init_checkpoint)
    if use_tpu:

      def tpu_scaffold():
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
        return tf.train.Scaffold()

      scaffold_fn = tpu_scaffold
    else:
      tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    all_layers = model.get_all_encoder_layers()

    predictions = {
        "unique_id": unique_ids,
    }

    for (i, layer_index) in enumerate(layer_indexes):
      predictions["layer_output_%d" % i] = all_layers[layer_index]

    output_spec = tf.contrib.tpu.TPUEstimatorSpec(
        mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    return output_spec

  return model_fn

In [14]:
def convert_examples_to_features(examples, seq_length, tokenizer):
  """Loads a data file into a list of `InputBatch`s."""

  features = []
  for (ex_index, example) in enumerate(examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
      tokens_b = tokenizer.tokenize(example.text_b)

    if tokens_b:
      # Modifies `tokens_a` and `tokens_b` in place so that the total
      # length is less than the specified length.
      # Account for [CLS], [SEP], [SEP] with "- 3"
      _truncate_seq_pair(tokens_a, tokens_b, seq_length - 3)
    else:
      # Account for [CLS] and [SEP] with "- 2"
      if len(tokens_a) > seq_length - 2:
        tokens_a = tokens_a[0:(seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0     0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    input_type_ids = []
    tokens.append("[CLS]")
    input_type_ids.append(0)
    for token in tokens_a:
      tokens.append(token)
      input_type_ids.append(0)
    tokens.append("[SEP]")
    input_type_ids.append(0)

    if tokens_b:
      for token in tokens_b:
        tokens.append(token)
        input_type_ids.append(1)
      tokens.append("[SEP]")
      input_type_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < seq_length:
      input_ids.append(0)
      input_mask.append(0)
      input_type_ids.append(0)

    assert len(input_ids) == seq_length
    assert len(input_mask) == seq_length
    assert len(input_type_ids) == seq_length

    if ex_index < 5:
      tf.logging.info("*** Example ***")
      tf.logging.info("unique_id: %s" % (example.unique_id))
      tf.logging.info("tokens: %s" % " ".join(
          [tokenization.printable_text(x) for x in tokens]))
      tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
      tf.logging.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
      tf.logging.info(
          "input_type_ids: %s" % " ".join([str(x) for x in input_type_ids]))

    features.append(
        InputFeatures(
            unique_id=example.unique_id,
            tokens=tokens,
            input_ids=input_ids,
            input_mask=input_mask,
            input_type_ids=input_type_ids))
  return features

def _truncate_seq_pair(tokens_a, tokens_b, max_length):
  """Truncates a sequence pair in place to the maximum length."""

  # This is a simple heuristic which will always truncate the longer sequence
  # one token at a time. This makes more sense than truncating an equal percent
  # of tokens from each, since if one sequence is very short then each token
  # that's truncated likely contains more information than a longer sequence.
  while True:
    total_length = len(tokens_a) + len(tokens_b)
    if total_length <= max_length:
      break
    if len(tokens_a) > len(tokens_b):
      tokens_a.pop()
    else:
      tokens_b.pop()

In [15]:
# def read_examples(input_file):
#   """Read a list of `InputExample`s from an input file."""
#   examples = []
#   unique_id = 0
#   # with tf.gfile.GFile(input_file, "r") as reader:
#   with open(input_file, "r") as reader:
#     while True:
#       # 将每行转成unicode
#       line = tokenization.convert_to_unicode(reader.readline())
#       if not line:
#         break
#       line = line.strip()
#       text_a = None
#       text_b = None
#       # 以『|||』进行分隔，前面是句子A，后面是句子B
#       m = re.match(r"^(.*) \|\|\| (.*)$", line)
#       if m is None:
#         text_a = line
#       else:
#         text_a = m.group(1)
#         text_b = m.group(2)
#       # 使用InputExample类封装一下
#       examples.append( 
#           InputExample(unique_id=unique_id, text_a=text_a, text_b=text_b))
#       unique_id += 1
#   return examples

def read_examples(lines):
  examples = []
  unique_id = 0
  for line in lines:
    line = tokenization.convert_to_unicode(line)
    if not line:
      continue
    line = line.strip()
    text_a = None
    text_b = None
    # 以『|||』进行分隔，前面是句子A，后面是句子B
    m = re.match(r"^(.*) \|\|\| (.*)$", line)
    if m is None:
      text_a = line
    else:
      text_a = m.group(1)
      text_b = m.group(2)
    # 使用InputExample类封装一下
    examples.append(
        InputExample(unique_id=unique_id, 
            text_a=text_a, text_b=text_b))
    unique_id += 1
  return examples

In [16]:
layer_indexes = LAYERS

bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

tokenizer = tokenization.FullTokenizer(
      vocab_file=VOCAB_FILE, do_lower_case=True)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    tpu_config=tf.contrib.tpu.TPUConfig(
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=is_per_host))

#---------------------------------------------

model_fn = model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=INIT_CHECKPOINT,
    layer_indexes=layer_indexes,
    use_tpu=True,
    use_one_hot_embeddings=True)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=True,
    model_fn=model_fn,
    config=run_config,
    predict_batch_size=BATCH_SIZE,
    train_batch_size=BATCH_SIZE)



def get_features(input_text, dim=768):


  examples = read_examples(input_text)

  features = convert_examples_to_features(
      examples=examples, seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)

  unique_id_to_feature = {}
  for feature in features:
    unique_id_to_feature[feature.unique_id] = feature

 
  input_fn = input_fn_builder(
      features=features, seq_length=MAX_SEQ_LENGTH)

  # Get features

  output_all = []

  for result in estimator.predict(input_fn, yield_single_examples=True):
    unique_id = int(result["unique_id"])
    feature = unique_id_to_feature[unique_id]
    output = collections.OrderedDict()
    for (i, token) in enumerate(feature.tokens):
      layers = []
      for (j, layer_index) in enumerate(layer_indexes):
        layer_output = result["layer_output_%d" % j]
        layer_output_flat = np.array([x for x in layer_output[i:(i + 1)].flat])
        layers.append(layer_output_flat)
      output[token] = sum(layers)[:dim]
    output_all.append(output)

  return output_all


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpemvryq5k', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.29.32.34:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff255b9c190>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.29.32.34:8470', '_evaluation_master': 'grpc://10.29.32.34:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker

In [ ]:
# embeddings = get_features("/content/drive/MyDrive/Explained_NLP_data/text/0.txt", dim=20)

n = len(subdoc)
print('number of docs: {}'.format(n))
results = get_features(subdoc)

# for file in tqdm(files):
#   embeddings = get_features(file, dim=768)
#   cls = embeddings['[CLS]'].reshape((1,768))
#   doc_matrix = np.append(doc_matrix, cls)
#   print(doc_matrix.shape)

# with open("/content/text2.csv",'ab') as f:
#   np.savetxt(f, doc_matrix, delimiter=",")

# # for file in files:
# for i in range(500):
#   input_file="/content/drive/MyDrive/explained nlp/text2/r" + str(i) + ".txt" 
#   embeddings = get_features(input_file, dim=768)
#   doc_matrix=np.r_[doc_matrix,embeddings['[CLS]'].reshape((1,1024))]
#   CLS = embeddings['[CLS]'].reshape((1,768))
#   with open("/content/drive/MyDrive/temp2.csv",'ab') as f:
#     np.savetxt(f, CLS, delimiter=",")



number of docs: 100000
INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: 0
INFO:tensorflow:tokens: [CLS] ( well not really 16 ##u as much as 6 + 4 + 6 , but it works if you ' re using short components . ) for ras ##t racks you use 6 ##u rails , but if you get two ( and flip the bottom one upside down ) you can stick these in between for another 4 ##u . i used little rubber feet for some extra space to make sure there was enough clearance . [SEP]
INFO:tensorflow:input_ids: 101 1006 2092 2025 2428 2385 2226 2004 2172 2004 1020 1009 1018 1009 1020 1010 2021 2009 2573 2065 2017 1005 2128 2478 2460 6177 1012 1007 2005 20710 2102 27259 2017 2224 1020 2226 15168 1010 2021 2065 2017 2131 2048 1006 1998 11238 1996 3953 2028 14961 2091 1007 2017 2064 6293 2122 1999 2090 2005 2178 1018 2226 1012 1045 2109 2210 8903 2519 2005 2070 4469 2686 2000 2191 2469 2045 2001 2438 14860 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensor

In [18]:
embeddings = np.array([result['[CLS]'] for result in results])
np.save("text2-2.npy", embeddings)

In [19]:
np.save("/content/drive/MyDrive/text2-2.npy", embeddings)

In [7]:
import numpy as np

In [11]:
embeddings = np.load('/content/drive/MyDrive/text2.npy')
print(embeddings.shape)

(99912, 768)


In [ ]:
#there are 99912 records in the first 100000

In [22]:
print(subdoc[0])

The portfolio is fine except for the fact that the last movement of sonata #6 is missing. What should one expect?


## SVD

In [17]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random


svd = TruncatedSVD(n_components=128, n_iter=7, random_state=42)
svd.fit(embeddings)
reduction_matrix = svd.transform(embeddings)

# print(svd.explained_variance_ratio_)

# print(svd.explained_variance_ratio_.sum())

# print(svd.singular_values_)

In [18]:
print(reduction_matrix.shape)

(99912, 128)


## PCA



In [ ]:
import numpy as np
from sklearn.decomposition import PCA
print("the following is the original matrix")
print(doc_matrix)
pca = PCA(n_components=4)  ##样本数要大于128
pca.fit(doc_matrix)

# print(pca.components_)
print("dimension reduction results")
reduction_matrix = pca.transform(doc_matrix)
print(reduction_matrix)

#print(pca.explained_variance_ratio_)

# print(pca.singular_values_)


## A linear layer link 128 to 1280

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as f
torch.manual_seed(2021)

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(4, 1280)  # 

    def forward(self, x):

        # If the size is a square you can only specify a single number
        x = x.view(-1, 4)
        x = f.relu(self.fc1(x))
        return x

net = Net()
print(net)

In [ ]:

input = torch.from_numpy(reduction_matrix).reshape(5,1,4).float()
out = net(input)
print(out)

# gpt-2


In [25]:
! pip install transformers

     |████████████████████████████████| 2.1MB 6.5MB/s 
     |████████████████████████████████| 901kB 30.9MB/s 
     |████████████████████████████████| 3.3MB 29.0MB/s 


In [26]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large',bos_token='<|startoftext|>',eos_token='<|endoftext|>',pad_token='<|pad|>')

model_gpt2 = GPT2LMHeadModel.from_pretrained('gpt2-large')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [30]:
y = out.reshape(5,1,1280) 

NameError: ignored

In [ ]:
# 得到原始文本的token array
raw_token = np.empty((0,max_len)) ## 文本限定50词
file_path= "/content/drive/MyDrive/text"
files= os.listdir(file_path) #得到文件夹下的所有文件名称
for file in files: #遍历文件夹
     if not os.path.isdir(file): #判断是否是文件夹，不是文件夹才打开
          input_file="/content/drive/MyDrive/text/"+file
          f = open(input_file)
          text = f.read()         
          labels = tokenizer.encode(text) # 
          text_len = len(labels)
          if text_len < max_len:
            labels = list(labels + [0] * (max_len - text_len))  ### 长度不够的补0
          raw_token = np.append(raw_token,np.array(labels).reshape(1,max_len), axis=0)

In [27]:
# 得到原始文本的token array
# import os

# file_path= "/content/drive/MyDrive/Explained_NLP_data/text"
raw_token = []
# max_len = 0
# files= os.listdir(file_path) #得到文件夹下的所有文件名称
for i in subdoc:
  # input_file="/content/drive/MyDrive/Explained_NLP_data/text/"+file
  # f = open(input_file)
  # text = f.read()         
  labels = tokenizer.encode(i) # 
  raw_token.append(labels)


          # raw_token = np.append(raw_token,np.array(labels).reshape(1,max_len), axis=0)

Token indices sequence length is longer than the specified maximum sequence length for this model (1907 > 1024). Running this sequence through the model will result in indexing errors


In [31]:
print(len(raw_token))

100000


In [ ]:
for m in range(len(y)):
  line = y[m].unsqueeze(0)
  output = model_gpt2(inputs_embeds = line)
  logits = output.logits
  batch_size, seq_length, hidden_size = logits.size()
  token = torch.argmax(logits[0][seq_length-1]) # the last token id for the first sample in the batch 
  generated = [token.item()]

  x = raw_token[m]
  N = max_len  # N more tokens to generate
  p_dropout = 0.1
  for i in range(N):
    wte_vector = model_gpt2.transformer.wte.weight[generated,:] # word embedding
    wpe_vector = model_gpt2.transformer.wpe.weight[len(generated),:] # positional embedding
    vector = wte_vector + f.dropout(wpe_vector,p_dropout)    ##
    vector = vector.unsqueeze(0)  ##
    
    labels_list = x[0:i+1]
    labels_list = torch.from_numpy(np.array(labels_list)).float()

    output = model_gpt2(inputs_embeds = vector,labels = labels_list.long())
    logits = output.logits
    batch_size, seq_length, hidden_size = logits.size()
    
    token = torch.argmax(logits[0][seq_length-1]) # the last token id for the first sample in the batch 
    generated += [token.item()]

  generated_seq = tokenizer.decode(generated)
  print(generated_seq)

In [ ]:
loss_list = []
for m in range(1,len(y)):
  import torch.nn.functional as f
  line = y[m].unsqueeze(0)
  output = model_gpt2(inputs_embeds = line)
  logits = output.logits
  batch_size, seq_length, hidden_size = logits.size()
  token = torch.argmax(logits[0][seq_length-1]) # the last token id for the first sample in the batch 
  generated = [token.item()]

  x = raw_token[m]
  N = len(x)  # N more tokens to generate
  p_dropout = 0.1
  for i in range(N):
    wte_vector = model_gpt2.transformer.wte.weight[generated,:] # word embedding
    wpe_vector = model_gpt2.transformer.wpe.weight[len(generated),:] # positional embedding
    vector = wte_vector + f.dropout(wpe_vector,p_dropout)    ##
    vector = vector.unsqueeze(0)  ##
    
    labels_list = x[0:i+1]
    labels_list = torch.from_numpy(np.array(labels_list)).float()

    output = model_gpt2(inputs_embeds = vector,labels = labels_list.long())
    loss = output.loss
    
    loss_list.append(loss.item())
    logits = output.logits
    batch_size, seq_length, hidden_size = logits.size()
    
    token = torch.argmax(logits[0][seq_length-1]) # the last token id for the first sample in the batch 
    generated += [token.item()]

  generated_seq = tokenizer.decode(generated)
  print(generated_seq)
  file_path = "/content/drive/MyDrive/generated/"+ str(m) + ".txt"
  with open(file_path,'a') as f:
    f.write(generated_seq)